# Segmenting and Clustering Neighborhoods in Toronto
______________________

### Week 3 Capstone Project

### My apologies, my pandas library is weird, I dont know how to fix it. It is not printing well dataframes. It works, but display is not working well.

In [1]:
import pandas as pd

In [2]:
import numpy as np # library to handle data in a vectorized manner

#import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The fo

In [3]:
Source = "/resources/Sheesh/Capstone/W3/FWD.csv"
File = open(Source, "r")

In [4]:
df = pd.read_csv(Source)

In [5]:
df.head()

ImportError: cannot import name 'OrderedDict'

  Postcode      Borough                            Neighborhood   Latitude  \
0      M1B  Scarborough                          Malvern, Rouge  43.806686   
1      M1C  Scarborough  Highland Creek, Port Union, Rouge Hill  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  

In [6]:
print('The df has {} boroughs and {} postal codes.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The df has 11 boroughs and 103 postal codes.


In [9]:
print('Shape:', df.shape)

Shape: (103, 5)


In [7]:
df.loc[8, 'Postcode']

'M1M'

In [8]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="texp")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [9]:
mapt = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='1066CC',
        fill_opacity=0.9,
        parse_html=False).add_to(mapt)  
    
mapt

In [11]:
# Fourdquare Credentials
CLIENT_ID = '0GE21IKRH3FZODF1RNWDMLN1LUFENFD0EAYCZ3VXT4V4OMAE' # your Foursquare ID
CLIENT_SECRET = '0EO1LG0JP10J3CN5HYFDCCZM3ACMDXPZWPBY1WXQBIQZHXAI' # your Foursquare Secret
VERSION = '20200320' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0GE21IKRH3FZODF1RNWDMLN1LUFENFD0EAYCZ3VXT4V4OMAE
CLIENT_SECRET:0EO1LG0JP10J3CN5HYFDCCZM3ACMDXPZWPBY1WXQBIQZHXAI


In [13]:
df.loc[3, 'Neighborhood']

'Woburn'

In [14]:
neighborhood_latitude = df.loc[3, 'Latitude']
neighborhood_longitude = df.loc[3, 'Longitude'] 

neighborhood_name = df.loc[3, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Woburn are 43.7709921, -79.21691740000001.


In [15]:
# Get request URL
Limit = 20
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    Limit)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=0GE21IKRH3FZODF1RNWDMLN1LUFENFD0EAYCZ3VXT4V4OMAE&client_secret=0EO1LG0JP10J3CN5HYFDCCZM3ACMDXPZWPBY1WXQBIQZHXAI&v=20200320&ll=43.7709921,-79.21691740000001&radius=500&limit=20'

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e745df2006dce001bffb79f'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7754921045, 'lng': -79.21069729639068},
   'sw': {'lat': 43.7664920955, 'lng': -79.22313750360935}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4cc1d28c06c254815ac18547',
       'name': 'Starbucks',
       'location': {'address': '300 Borough Dr',
        'crossStreet': 'Scarborough Town Centre',
        'lat': 43.770037201625215,
        'lng': -79.22115586641958,
        'labeledLatLngs':

In [17]:
# Define get_category_type function
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


AttributeError: module 'pandas.compat' has no attribute 'string_types'

In [19]:
nearby_venues.head()

ImportError: cannot import name 'OrderedDict'

           venue.name                                   venue.categories  \
0           Starbucks  [{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...   
1         Tim Hortons  [{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...   
2  Korean Grill House  [{'id': '4bf58dd8d48988d113941735', 'name': 'K...   

   venue.location.lat  venue.location.lng  
0           43.770037          -79.221156  
1           43.770827          -79.223078  
2           43.770812          -79.214502  

In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
Toronto_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'])

Malvern, Rouge
Highland Creek, Port Union, Rouge Hill
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South, Flemingdon Park
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
Riverdale, The Danf

In [23]:
# Size of the new df
print(Toronto_venues.shape)
Toronto_venues.head()

(1077, 7)


ImportError: cannot import name 'OrderedDict'

                             Neighborhood  Neighborhood Latitude  \
0                          Malvern, Rouge              43.806686   
1                          Malvern, Rouge              43.806686   
2  Highland Creek, Port Union, Rouge Hill              43.784535   
3  Highland Creek, Port Union, Rouge Hill              43.784535   
4       Guildwood, Morningside, West Hill              43.763573   

   Neighborhood Longitude                      Venue  Venue Latitude  \
0              -79.194353                    Wendy's       43.807448   
1              -79.194353      Interprovincial Group       43.805630   
2              -79.160497      Royal Canadian Legion       43.782533   
3              -79.160497  Affordable Toronto Movers       43.787919   
4              -79.188711          G & G Electronics       43.765309   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.200378            Print Shop  
2       -79.163085           

In [28]:
# Count of venues by neighborhood
Toronto_venues.groupby('Neighborhood').count()

ImportError: cannot import name 'OrderedDict'

                                                    Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                               20   
Agincourt                                                               4   
Agincourt North, L'Amoreaux East, Milliken, Ste...                      3   
Albion Gardens, Beaumond Heights, Humbergate, J...                     10   
Alderwood, Long Branch                                                  9   
Bathurst Manor, Downsview North, Wilson Heights                        20   
Bathurst Quay, CN Tower, Harbourfront West, Isl...                     16   
Bayview Village                                                         4   
Bedford Park, Lawrence Manor East                                      20   
Berczy Park                                                            20   
Birch Cliff, Cliffside West                                             4   

In [30]:
# Unique categories from venues
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 214 uniques categories.


In [31]:
# Analysis by neighborhood
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

ImportError: cannot import name 'OrderedDict'

   Yoga Studio  Accessories Store  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Art Gallery  Arts & Crafts Store  Asian Restaurant  A

In [32]:
Toronto_onehot.shape

(1077, 214)

In [33]:
# Group by mean frequency of ocurrence 
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

ImportError: cannot import name 'OrderedDict'

                                         Neighborhood  Yoga Studio  \
0                            Adelaide, King, Richmond     0.000000   
1                                           Agincourt     0.000000   
2   Agincourt North, L'Amoreaux East, Milliken, St...     0.000000   
3   Albion Gardens, Beaumond Heights, Humbergate, ...     0.000000   
4                              Alderwood, Long Branch     0.000000   
5     Bathurst Manor, Downsview North, Wilson Heights     0.000000   
6   Bathurst Quay, CN Tower, Harbourfront West, Is...     0.000000   
7                                     Bayview Village     0.000000   
8                   Bedford Park, Lawrence Manor East     0.000000   
9                                         Berczy Park     0.000000   
10                        Birch Cliff, Cliffside West     0.000000   
11  Bloordale Gardens, Eringate, Markland Wood, Ol...     0.000000   
12       Brockton, Exhibition Place, Parkdale Village     0.000000   
13  Business Reply M

In [35]:
Toronto_grouped.shape

(99, 214)

In [36]:
# Top 5 most common venues by neighborhood 
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0             Hotel  0.10
1  Greek Restaurant  0.05
2        Steakhouse  0.05
3        Food Court  0.05
4            Lounge  0.05


----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1  Latin American Restaurant  0.25
2                     Lounge  0.25
3             Clothing Store  0.25
4                      Motel  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                        venue  freq
0                        Park  0.33
1                 Coffee Shop  0.33
2                  Playground  0.33
3  Modern European Restaurant  0.00
4                   Locksmith  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store   0.2
1           Pizza Place   0.2
2  Fast Food Restaurant   0.1
3            Beer Store   0.1
4        Sandwich Place   0.

In [40]:
# Sort
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
# Print top 10 most common venues by neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

ImportError: cannot import name 'OrderedDict'

                                        Neighborhood  \
0                           Adelaide, King, Richmond   
1                                          Agincourt   
2  Agincourt North, L'Amoreaux East, Milliken, St...   
3  Albion Gardens, Beaumond Heights, Humbergate, ...   
4                             Alderwood, Long Branch   

       1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0                      Hotel      Greek Restaurant      Asian Restaurant   
1  Latin American Restaurant        Breakfast Spot        Clothing Store   
2                Coffee Shop            Playground                  Park   
3              Grocery Store           Pizza Place  Fast Food Restaurant   
4                Pizza Place           Coffee Shop        Sandwich Place   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0            Restaurant    Seafood Restaurant                Lounge   
1                Lounge    Dim Sum Restaurant  Ethiopian Restaur

## Thank you for reading :D